This notebook is based on the [Lagrangian NN tutorial](https://colab.research.google.com/drive/1CSy-xfrnTX28p1difoTA8ulYw0zytJkq).

## The double pendulum

In [1]:
import jax

In [2]:
jax.__version__

'0.6.1'

In [3]:
# For the creation of images
#!pip install -U -q Pillow moviepy proglog

In [4]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.experimental.ode import odeint

import matplotlib.pyplot as plt
from functools import partial # reduces arguments to function by making some subset implicit

from jax.example_libraries import stax, optimizers


# visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from moviepy import ImageSequenceClip
from functools import partial
import proglog
from PIL import Image

In [8]:
# Making sure we are using GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

"""
/tmp/ipykernel_38924/3960750258.py:3: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)""";


gpu


One can follow the [derivation done by Diego Assencio](https://dassencio.org/33) on the Lagrangian formulation of the double pendulum. From this, one observes that the Lagrangian formualtion of the double pendulum becomes

$$
    \mathcal{L} = \frac{1}{2}\left( m_1 + m_2 \right)l^2_1\dot{\theta}^2_1 + \frac{1}{2}m_2l^2_2\dot{\theta}^2_2 + m_2l_1l_2\dot{\theta}_1\dot{\theta}_2\cos(\theta_1-\theta_2) + \left(m_1+m_2\right)gl_1\cos(\theta_1) + m_2gl_2\cos(\theta_2) 
$$

$$
    \mathcal{L} = \frac{1}{2}m_1 l^2_1\dot{\theta}^2_1 + \frac{1}{2}m_2\left(l^2_1\dot{\theta}^2_1 + l^2_2\dot{\theta}^2_2 + 2l_1l_2\dot{\theta}_1\dot{\theta}_2\cos(\theta_1-\theta_2)\right) + \left(m_1+m_2\right)gl_1\cos(\theta_1) + m_2gl_2\cos(\theta_2) 
$$

Where all the terms that do not contain the gravity $g$ belong to the kinetic energy. To obtain the canonical momenta $p_{\theta_j}$ associates with the coordinate $\theta_j$, one just computes $\partial \mathcal{L} / \partial \dot{\theta}_j$.

In [9]:
# Compute the Lagrangian
def get_Lagrangian(q, q_dot, m1=1, m2=1, l1=1, l2=1, g=9.8):
    θ_1, θ_2 = q
    θ_dot_1, θ_dot_2 = q_dot
    
    E_kin_1 = 0.5*m1*jnp.square(l1*θ_1)
    E_kin_2 = 0.5*m2*( jnp.square(l1*θ_1) + jnp.square(l2*θ_2) + 2*l1*l2*θ_dot_1*θ_dot_2*jnp.cos(θ_1-θ_2) )
    
    T = E_kin_1 + E_kin_2
    V = -g*m2*( l1*jnp.cos(θ_1) +  l2*jnp.cos(θ_2) ) - g*m1*l1*jnp.cos(θ_1)
    return T - V

It is possible to assess for the quality of the Lagrangian-NN as there's an analytical solution for this problem. By following Diego's derivation, one observes that

$$
    \displaystyle\frac{d}{dt}
    \left( \begin{matrix} \theta_1 \\[1pt] \theta_2 \\[1pt] \omega_1 \\[1pt] \omega_1 \end{matrix} \right)
    =
    \left( \begin{matrix} \omega_1 \\ \omega_2 \\ g_1(\theta_1,\theta_2,\omega_1,\omega_2)
    \\ g_2(\theta_1,\theta_2,\omega_1,\omega_2) \end{matrix} \right),
$$

where

$$
    g_1 := \displaystyle\frac{f_1 - \alpha_1 f_2}{1 - \alpha_1\alpha_2}
    \quad\quad
    g_2 := \displaystyle\frac{f_2 - \alpha_2 f_1}{1 - \alpha_1\alpha_2}
$$

and

$$
    \begin{align}
    & \alpha_1(\theta_1,\theta_2) ~:=~ \displaystyle\frac{l_2}{l_1}\left(\frac{m_2}{m_1 + m_2}\right)\cos(\theta_1 - \theta_2)\\
    &\alpha_2(\theta_1,\theta_2) ~:=~ \frac{l_1}{l_2}\cos(\theta_1-\theta_2)\\
    &\displaystyle f_1(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) ~:=~
    -\frac{l_2}{l_1}\left(\frac{m_2}{m_1+m_2}\right) \dot{\theta}_2^2\sin(\theta_1 - \theta_2)
    - \frac{g}{l_1} \sin\theta_1 \\
    &\displaystyle f_2(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) ~:=~
    \frac{l_1}{l_2}\dot{\theta}_1^2\sin(\theta_1-\theta_2) - \frac{g}{l_2} \sin\theta_2 
    \end{align}
$$

These functions will also be computed to then create the loss function. 


In [ ]:
def get_analytical_sol(state, m1=1, m2=1, l1=1, l2=1, g=9.8):
    θ_1, θ_2, θ_dot_1, θ_dot_2 = state
    
    α1 = (l2/l1)*(m2/(m1+m2))*jnp.cos(θ_1-θ_2)
    α2 = (l1/l2)*jnp.cos(θ_1-θ_2)
    
    f1 = -(l2/l1)*(m2/(m1+m2))*jnp.square(θ_dot_2)*jnp.sin(θ_1-θ_2) - g*jnp.sin(θ_1)/l1
    f2 = (l1/l2)*jnp.square(θ_dot_1)*jnp.sin(θ_1-θ_2) - g*jnp.sin(θ_2)/l2
    
    g1 = (f1 - α1*f2)/(1 - α1*α2)
    g2 = (f2 - α2*f1)/(1 - α1*α2)
    
    return jnp.stack([θ_dot_1,θ_dot_2,g1,g2])

The next step is to obtain the dynamics of the system with numerical integration. We take the vectorized form of the Euler-Lagrange equation 

$$
    \ddot q = (\nabla_{\dot q}\nabla_{\dot q}^{\top}\mathcal{L})^{-1}[\nabla_q \mathcal{L} - (\nabla_{q}\nabla_{\dot q}^{\top}\mathcal{L})\dot q]
$$

It is possible to notice that the neural network will depend on the inverse of a Hessian matrix during the forward pass. This is not an easy implementation; however, the authors of the research propose the usage of JAX, which can implement the previous equation straightforwardly. 

In [ ]:
def get_eqs_of_motion(L, t, state):
    q, q_t = jnp.split(state,2)
    q_tt =  ( jnp.linalg.pinv( jax.hessian(L,1)(q,q_t) ) @ 
              jax.grad(L,1)(q,q_t) - jax.jacobian(jax.jacobian(L,0)(q,q_t) @ q_t)
            )
    return jnp.concatenate([q_t, q_tt])

def solve_lagrangian(L, initial_state, **kwargs):
  # We currently run odeint on CPUs only, because its cost is dominated by
  # control flow, which is slow on GPUs.
  @partial(jax.jit, backend='cpu')
  def f(initial_state):
    return odeint(partial(get_eqs_of_motion, L),
                  initial_state, **kwargs)
  return f(initial_state)